In [1]:
import anndata as ad
import scanpy as sc
import scanpy.external as sce

***Pre-process the data with scanpy***

In [3]:
adata = ad.read_h5ad("/well/sansom/users/odq545/work/cellhub/ifnb_example_full/integration.dir/anndata.dir/gex.h5ad")

In [4]:

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
# sc.pl.violin(adata, ['ngenes', 'total_UMI', 'pct_mitochondrial'],
#             jitter=0.4, multi_panel=True)

In [5]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers["log1p"] = adata.X.copy()

In [6]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=3000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="condition"
)

In [7]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)

***Perform Integration with Harmony***

In [8]:
sce.pp.harmony_integrate(adata, 'condition')

2022-07-15 01:41:00,642 - harmonypy - INFO - Iteration 1 of 10
2022-07-15 01:41:33,193 - harmonypy - INFO - Iteration 2 of 10
2022-07-15 01:42:34,918 - harmonypy - INFO - Iteration 3 of 10
2022-07-15 01:43:19,596 - harmonypy - INFO - Iteration 4 of 10
2022-07-15 01:43:52,391 - harmonypy - INFO - Iteration 5 of 10
2022-07-15 01:44:00,845 - harmonypy - INFO - Iteration 6 of 10
2022-07-15 01:44:14,139 - harmonypy - INFO - Converged after 6 iterations


In [11]:
adata

AnnData object with n_obs × n_vars = 29793 × 17865
    obs: 'library_id', 'barcode_id', 'ngenes', 'total_UMI', 'pct_mitochondrial', 'pct_ribosomal', 'pct_immunoglobin', 'pct_hemoglobin', 'library_id:1', 'scrub_doublet_scores', 'scrub_predicted_doublets', 'singleR_HPCA', 'singleR_BlueEnc', 'singleR_ImmCell', 'singleR_NovHem', 'singleR_MonImm', 'sample_id', 'cell_type', 'condition', 'replicate', 'genotype', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches', 'mean', 'std'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca', 'X_pca_harmony'
    varm: 'PCs'
    layers: 'counts', 'log1p'

In [13]:
adata.obsm['X_pca_harmony'].shape

(29793, 50)

In [12]:
adata.write_h5ad("integrated.h5ad")